<a href="https://colab.research.google.com/github/correctchemist/code/blob/main/bb1_dihydroorotate_part3b_descriptors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bioinformatics Project - Dihydroorotate dehydrogenase [Part 3b] Descriptor Calculation and Dataset Preparation**

Beatrice Iwuala

# Calculating molecular descriptors using RDkit and Mordred 

In **Part 3**, we will be calculating molecular descriptors that are essentially quantitative description of the compounds in the dataset. Finally, we will be preparing this into a dataset for subsequent model building in Part 4.

In [ ]:
!pip install rdkit-pypi
!pip install mordred

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.5 MB 1.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 128 kB 7.0 MB/s 
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176725 sha256=5404b9ee39222b7a8e6b35b058dd45e2e161d11116e01820bc470562d9e29b0e
  Stored in directory: /root/.cache/pip/wheels/02/c0/2e/e7e3d63b431777712ebc128bc4deb9ac5cb19afc7c1ea341ec
Successfully built mordred


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# https://www.rdkit.org/
#https://github.com/rdkit/rdkit
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

# https://pandas.pydata.org
import pandas as pd

# https://numpy.org/doc/stable/release.html
import numpy as np

#https://github.com/mordred-descriptor/mordred
from mordred import Calculator, descriptors

In [ ]:
# import session_info
# session_info.show()

In [36]:
dataset = pd.read_csv('bioactivity_preprocessed_data1.csv')

In [37]:
dataset.head()

,molecule_chembl_id,SMILES,bioactivity_class,standard_value
0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,42600.0
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,142600.0
2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,93400.0
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,153500.0
4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,200000.0


## 1. Generate canonical SMILES

In [38]:
# There might be one or more valid SMILES that can represent one compound
# Thanks to Pat Walters for this information,checkout his excellent blog: https://www.blogger.com/profile/18223198920629617711
def canonical_smiles(smiles):
    mols = [Chem.MolFromSmiles(smi) for smi in smiles] 
    smiles = [Chem.MolToSmiles(mol) for mol in mols]
    return smiles

In [39]:
# Canonical SMILES
Canon_SMILES = canonical_smiles(dataset.SMILES)
len(Canon_SMILES)

602

In [40]:
# Put the smiles in the dataframe
dataset['SMILES'] = Canon_SMILES
dataset

,molecule_chembl_id,SMILES,bioactivity_class,standard_value
0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,42600.0
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,142600.0
2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,93400.0
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,153500.0
4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,200000.0
...,...,...,...,...
597,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0
598,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0
599,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,250000.0
600,CHEMBL1956285,Cc1cc(Nc2ccc(S(F)(F)(F)(F)F)cc2)n2nc(C(C)(F)F)...,active,10.0


In [41]:
# Create a list for duplicate smiles
duplicates_smiles = dataset[dataset['SMILES'].duplicated()]['SMILES'].values
len(duplicates_smiles)

51

In [42]:
# Create a list for duplicate smiles
dataset[dataset['SMILES'].isin(duplicates_smiles)].sort_values(by=['SMILES'])

,molecule_chembl_id,SMILES,bioactivity_class,standard_value
97,CHEMBL973,C/C(O)=C(\C#N)C(=O)Nc1ccc(C(F)(F)F)cc1,inactive,190100.00
419,CHEMBL973,C/C(O)=C(\C#N)C(=O)Nc1ccc(C(F)(F)F)cc1,intermediate,3900.00
7,CHEMBL973,C/C(O)=C(\C#N)C(=O)Nc1ccc(C(F)(F)F)cc1,inactive,190100.00
326,CHEMBL973,C/C(O)=C(\C#N)C(=O)Nc1ccc(C(F)(F)F)cc1,inactive,190546.07
304,CHEMBL973,C/C(O)=C(\C#N)C(=O)Nc1ccc(C(F)(F)F)cc1,inactive,11390.00
...,...,...,...,...
572,CHEMBL2012827,O=C(NC1CCCC1)c1ccc(-n2cnc3ccccc32)s1,intermediate,2506.00
333,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,154881.66
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,153500.00
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,142600.00


## 2.  Drop duplicate values

In [43]:
dataset_new = dataset.drop_duplicates(subset=['SMILES'])
len(dataset_new)

551

In [44]:
dataset_new

,molecule_chembl_id,SMILES,bioactivity_class,standard_value
0,CHEMBL199572,CN(C(=O)c1ccc(-c2ccccc2)cc1)c1ccccc1C(=O)O,inactive,42600.0
1,CHEMBL199574,O=C(Nc1ccccc1C(=O)O)c1ccc2cc(Br)ccc2c1,inactive,142600.0
2,CHEMBL372561,CN(C(=O)c1ccc2cc(Br)ccc2c1)c1ccccc1C(=O)O,inactive,93400.0
3,CHEMBL370865,O=C(Nc1ccccc1C(=O)O)c1ccc(-c2ccccc2)cc1,inactive,153500.0
4,CHEMBL199575,CN(C(=O)c1ccc2ccccc2c1)c1ccccc1C(=O)O,inactive,200000.0
...,...,...,...,...
596,CHEMBL4544292,CCOc1nn(C)c(COc2ccccc2)c1C(=O)O,inactive,250000.0
597,CHEMBL4569109,Cn1nc(OCC2CC2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0
598,CHEMBL4568957,Cn1nc(OCc2ccccc2)c(C(=O)O)c1COc1ccccc1,inactive,250000.0
599,CHEMBL4449622,Cn1nc(O)c(C(N)=O)c1COc1ccccc1,inactive,250000.0


## Calculate descriptors using RDkit

### a. General molecular descriptors-about 200 molecular descriptors

In [45]:
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles] 
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()
    
    Mol_descriptors =[]
    for mol in mols:
        # add hydrogens to molecules
        mol=Chem.AddHs(mol)
        # Calculate all 200 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names 

# Function call
Mol_descriptors,desc_names = RDkit_descriptors(dataset_new['SMILES'])

In [46]:
df_with_200_descriptors = pd.DataFrame(Mol_descriptors,columns=desc_names)
df_with_200_descriptors

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,13.744296,-3.693263,13.744296,0.327769,0.774008,331.371,314.235,331.120843,124,0,...,0,0,0,0,0,0,0,0,0,0
1,13.252877,-1.606550,13.252877,0.164514,0.710808,370.202,358.106,369.000055,114,0,...,0,0,0,0,0,0,0,0,0,0
2,13.727252,-3.617987,13.727252,0.218218,0.719488,384.229,370.117,383.015705,120,0,...,0,0,0,0,0,0,0,0,0,0
3,13.269921,-1.677155,13.269921,0.265269,0.753503,317.344,302.224,317.105193,118,0,...,0,0,0,0,0,0,0,0,0,0
4,13.699475,-3.640487,13.699475,0.278290,0.800108,305.333,290.213,305.105193,114,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,12.345880,-3.603915,12.345880,0.208836,0.874182,276.292,260.164,276.111007,106,0,...,0,0,0,0,0,0,0,0,0,0
547,12.561833,-3.705638,12.561833,0.243275,0.850289,302.330,284.186,302.126657,116,0,...,0,0,0,0,0,0,0,0,0,0
548,12.701226,-3.662697,12.701226,0.206424,0.715545,338.363,320.219,338.126657,128,0,...,0,0,0,0,0,0,0,0,0,0
549,12.240651,-3.423406,12.240651,0.052663,0.835805,247.254,234.150,247.095691,94,0,...,0,0,0,0,0,0,0,0,0,0


### b. Fingerprints

In [47]:
def morgan_fpts(data):
    Morgan_fpts = []
    for i in data:
        mol = Chem.MolFromSmiles(i) 
        fpts =  AllChem.GetMorganFingerprintAsBitVect(mol,2,2048)
        mfpts = np.array(fpts)
        Morgan_fpts.append(mfpts)  
    return np.array(Morgan_fpts)

In [48]:
Morgan_fpts = morgan_fpts(dataset_new['SMILES'])
Morgan_fpts.shape

(551, 2048)

In [49]:
Morgan_fingerprints = pd.DataFrame(Morgan_fpts,columns=['Col_{}'.format(i) for i in range(Morgan_fpts.shape[1])])
Morgan_fingerprints

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_2038,Col_2039,Col_2040,Col_2041,Col_2042,Col_2043,Col_2044,Col_2045,Col_2046,Col_2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
547,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
548,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
549,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculate descreptors using Mordred-1826 descriptors

In [50]:
def All_Mordred_descriptors(data):
    calc = Calculator(descriptors, ignore_3D=False)
    mols = [Chem.MolFromSmiles(smi) for smi in data]
    
    # pandas df
    df = calc.pandas(mols)
    return df

In [51]:
mordred_descriptors = All_Mordred_descriptors(dataset_new['SMILES'])

100%|██████████| 551/551 [04:05<00:00,  2.24it/s]


In [52]:
mordred_descriptors.shape

(551, 1826)

In [53]:
mordred_descriptors

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,19.286802,15.324880,1,0,32.835775,2.399926,4.799852,32.835775,1.313431,4.140286,...,10.078071,59.661128,331.120843,7.883830,1606,40,128.0,150.0,8.138889,5.583333
1,17.953468,14.119765,1,0,29.412608,2.399383,4.798767,29.412608,1.278809,4.064617,...,10.036750,57.374616,369.000055,10.542859,1252,36,120.0,140.0,7.638889,5.027778
2,18.689085,15.045539,1,0,30.890693,2.424396,4.848792,30.890693,1.287112,4.106864,...,10.141441,58.723797,383.015705,10.079361,1357,40,126.0,149.0,8.5,5.250000
3,18.551185,14.401457,1,0,31.361251,2.361257,4.722514,31.361251,1.306719,4.099456,...,9.966134,58.302608,317.105193,8.130902,1487,36,122.0,141.0,7.277778,5.361111
4,17.872588,14.530208,1,0,30.132789,2.417143,4.834286,30.132789,1.310121,4.064963,...,10.080838,57.461246,305.105193,8.029084,1192,38,120.0,142.0,7.638889,5.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,15.015651,13.345507,1,0,25.099637,2.441664,4.712963,25.099637,1.254982,3.903098,...,9.714323,67.391681,276.111007,7.669750,844,28,98.0,113.0,7.166667,4.611111
547,17.136972,14.464287,1,0,28.548946,2.44913,4.722079,28.548946,1.297679,4.046759,...,9.888323,75.420376,302.126657,7.553166,1119,30,114.0,133.0,6.777778,4.861111
548,19.258292,15.594432,1,0,32.533192,2.445624,4.725929,32.533192,1.301328,4.138024,...,9.938130,73.599999,338.126657,7.863411,1627,35,126.0,145.0,7.527778,5.611111
549,13.710828,12.330677,0,0,22.475483,2.426553,4.687433,22.475483,1.248638,3.806356,...,9.644652,64.990302,247.095691,7.970829,629,25,90.0,104.0,6.666667,4.027778
